In [2]:
import pandas as pd


In [3]:
import numpy as np


In [4]:
import re


In [6]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [7]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [11]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.3 MB/s eta 0:00:00


In [12]:
import faiss

In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

In [14]:
df = pd.read_csv('www_111mtk111_com_log.csv')

In [15]:
df.head()

,log
0,"49.77.142.7 - - [07/Oct/2023:16:56:11] ""GET /j..."
1,"255.177.169.126 - - [23/Jul/2023:13:57:36] ""PO..."
2,"130.232.2.255 - - [01/Feb/2023:06:38:23] ""DELE..."
3,"114.167.138.229 - - [10/Aug/2023:00:12:26] ""PO..."
4,"240.232.70.69 - - [01/Oct/2023:15:31:42] ""PUT ..."


In [16]:
log_pattern = re.compile(r'(\S+) (\S+) (\S+) \[(.*?)\] "(.*?)" (\d{3}) (\S+)')

In [18]:
df[['ip', 'identity', 'user', 'timestamp', 'request', 'status_code', 'size']] = df['log'].str.extract(log_pattern)
df.drop(columns=['log', 'size'], inplace=True)

In [19]:
df.head()

,ip,identity,user,timestamp,request,status_code
0,49.77.142.7,-,-,07/Oct/2023:16:56:11,GET /jackets/leather/reviews HTTP/1.1,500
1,255.177.169.126,-,-,23/Jul/2023:13:57:36,POST /jackets/leather/details HTTP/1.1,500
2,130.232.2.255,-,-,01/Feb/2023:06:38:23,DELETE /shoes/sandals/details HTTP/1.1,500
3,114.167.138.229,-,-,10/Aug/2023:00:12:26,POST /shirts/dressshirt/details HTTP/1.1,500
4,240.232.70.69,-,-,01/Oct/2023:15:31:42,PUT /shoes/boots/photos HTTP/1.1,301


In [20]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%b/%Y:%H:%M:%S')

In [22]:
df['timestamp'].head()

,timestamp
0,2023-10-07 16:56:11
1,2023-07-23 13:57:36
2,2023-02-01 06:38:23
3,2023-08-10 00:12:26
4,2023-10-01 15:31:42


In [23]:
df['combined'] = df['request'] + " | Status Code: " + df['status_code'] + " | IP: " + df['ip'] + " | Timestamp: " + df['timestamp'].astype(str)

In [24]:
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
combined_embeddings = model.encode(df['combined'].tolist())

In [26]:
index = faiss.IndexFlatL2(combined_embeddings.shape[1])

In [27]:
index.add(np.array(combined_embeddings))

In [28]:
print(f"FAISS index'te toplam {index.ntotal} vektör var.")

FAISS index'te toplam 2000 vektör var.


In [100]:
user_query = input("Sormak istediğiniz soruyu girin: ")

Sormak istediğiniz soruyu girin: Log dosyasında en sık alınan hata kodu nedir?


In [101]:
query_embedding = model.encode([user_query])

In [102]:
D, I = index.search(query_embedding, k=5)

In [103]:
similar_logs = df.iloc[I[0]]
print(similar_logs)

                  ip identity user           timestamp  \
1237    181.49.60.45        -    - 2023-08-11 21:10:51   
493    42.67.191.192        -    - 2023-05-24 17:28:59   
940    20.81.178.212        -    - 2023-02-27 22:49:39   
1936  113.149.36.108        -    - 2023-04-27 04:18:49   
329    65.230.182.79        -    - 2023-05-12 04:57:06   

                                      request status_code  \
1237  POST /accessories/hats/reviews HTTP/1.1         200   
493   POST /accessories/hats/reviews HTTP/1.1         200   
940   POST /accessories/hats/reviews HTTP/1.1         200   
1936  POST /accessories/hats/reviews HTTP/1.1         500   
329   POST /accessories/hats/reviews HTTP/1.1         500   

                                               combined  
1237  POST /accessories/hats/reviews HTTP/1.1 | Stat...  
493   POST /accessories/hats/reviews HTTP/1.1 | Stat...  
940   POST /accessories/hats/reviews HTTP/1.1 | Stat...  
1936  POST /accessories/hats/reviews HTTP/1.1 | Stat

In [104]:
training_data = [
    {"question": "Kaç adet status_code değeri 200’dür?", "answer": "200 adet status_code bulunmaktadır."},
    {"question": "En çok erişilen sayfa nedir?", "answer": "GET /index.html"},
    # Diğer soru-cevap çiftlerini buraya eklenebilir.
]

In [105]:
train_df = pd.DataFrame(training_data)

In [106]:
model_name = "t5-small"
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [107]:
def preprocess_data(data):
    input_texts = []
    target_texts = []
    for item in data:
        input_text = f"question: {item['question']} context: Log verileri"  # Log verileri yerine uygun bağlam ekleyebiliriz
        target_text = item['answer']
        input_texts.append(input_text)
        target_texts.append(target_text)
    return input_texts, target_texts

input_texts, target_texts = preprocess_data(training_data)

In [108]:
train_encodings = t5_tokenizer(input_texts, truncation=True, padding=True, max_length=512)
train_labels = t5_tokenizer(target_texts, truncation=True, padding=True, max_length=512)


In [109]:
train_dataset = {
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels["input_ids"]
}


In [110]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

In [111]:
import torch

In [112]:
from torch.utils.data import Dataset

In [113]:
class QADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [114]:
train_dataset = QADataset(train_encodings, train_labels["input_ids"])

In [115]:
trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=t5_tokenizer
)

In [116]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=6.019788106282552, metrics={'train_runtime': 7.4425, 'train_samples_per_second': 0.806, 'train_steps_per_second': 0.403, 'total_flos': 44409028608.0, 'train_loss': 6.019788106282552, 'epoch': 3.0})

In [117]:
log_text = " ".join(similar_logs['combined'].tolist())

In [118]:
input_text = f"question: {user_query} context: {log_text}"

In [119]:
input_ids = t5_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)


In [120]:
outputs = t5_model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)


In [121]:
answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Modelin cevabı: {answer}")

Modelin cevabı: POST /accessories/hats/reviews HTTP/1.1


In [122]:
if len(similar_logs) == 0:
    print("Sistemde bu soruya karşılık verecek bir veri yok.")
else:

    print(f"Modelin cevabı: {answer}")

Modelin cevabı: POST /accessories/hats/reviews HTTP/1.1
